In [14]:
# importing the required libraries
library(magick)
library(image.libfacedetection)
library(keras)

In [15]:
# loading a sample input image and resizing it
width_resized = 500
height_resized = 500

test_img = image_read("data/face_recognition/brad_pitt/brad_pitt_21.jpg")
test_img <- image_scale(test_img,paste0(width_resized,"x",height_resized,sep =""))

In [16]:
# localizing the face in the image
faces <- image_detect_faces(test_img)

In [17]:
# printing and saving the attributes of the face locale
faces$detections[,1:4]
face_width = faces$detections$width
face_height = faces$detections$height 
face_x = faces$detections$x
face_y = faces$detections$y

x,y,width,height
239,26,52,52


In [1]:
# drawing a bounding box around the face
test_img <- image_draw(test_img)
rect(xleft = face_x,ybottom = face_y,xright = face_x+ face_width,ytop = face_y+ face_height,lwd = 2,border = "red")
dev.off()
plot(test_img)

In [24]:
# preparing the training data by resizing, localizing and cropping each face in the data

fold = list.dirs('data/face_recognition',full.names = FALSE, recursive = FALSE)
fold = grep(paste0("face", collapse = "|"), fold, invert = TRUE, value = TRUE)

train_data_dir = "data/face_recognition/faces/"
dir.create(train_data_dir)

for (i in fold){
    files = list.files(path = paste0("data/face_recognition/",i),full.names = FALSE)
    for (face_file in files){
        img = image_read(paste0("data/face_recognition/",i,"/",face_file,sep= ""))
        img <- image_scale(img,paste0(width_resized,"x",height_resized,sep =""))
        faces <- image_detect_faces(img)
        face_width = faces$detections$width
        face_height = faces$detections$height
        face_x = faces$detections$x
        face_y = faces$detections$y
        face_dim= paste0(face_width,"x",face_height,"+",face_x,"+",face_y)
        face_cropped = image_crop(img,face_dim)
        face_cropped = image_crop(img,face_dim)
        if(nchar(face_dim)<= 3){
            print(paste("empty face in:",face_file))
            print(face_file)
        }else{
            fold_name = paste0(train_data_dir,"/",i,"_face")
            dir.create(fold_name)
            image_write(face_cropped,paste0(fold_name,"/",face_file))
                      
        }
        
    }
} 

In [8]:
# setting the size of the training images
train_path = "data/face_recognition/faces/"
img_size = c(160,160)
img_channels = 3

class_label = list.dirs('data/face_recognition/faces', full.names = FALSE, recursive = TRUE)[-1]

In [10]:
# building a training generator
train_data_generator <- image_data_generator(rotation_range = 10,shear_range = .2,rescale = 1/255,
                                            width_shift_range = 0.1,
                                            height_shift_range = 0.1,
                                            fill_mode = "nearest")

In [11]:
# getting the data
train_data <- flow_images_from_directory(directory = train_path,shuffle = T,
                                         generator = train_data_generator,
                                         target_size = img_size,
                                         color_mode = "rgb",
                                         class_mode = "categorical",
                                         classes = class_label,
                                         batch_size = 10)

In [12]:
# loading the FaceNet model
facenet <- load_model_hdf5("facenet_keras.h5")

print(facenet$input)
print(facenet$output)

In [15]:
# building our face recognition model
facenet_out <- facenet$output %>%
layer_dense(units = 128,activation = "relu") %>%
layer_dense(units = 3,activation = "softmax")

facenet_model <- keras_model(inputs = facenet$input, outputs = facenet_out)

In [17]:
# freezing the pre-realised imagenet weights of the FaceNet the model
freeze_weights(facenet)

In [18]:
# compiling the model
facenet_model %>% compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',metrics = c('accuracy'))

# training the model
facenet_model %>% fit_generator(generator = train_data,steps_per_epoch = 2,
                                 epochs = 5)

In [21]:
# saving the model
facenet_model %>% save_model_hdf5(paste0("facenet_model.h5",sep=""))

In [31]:
# recognizing face from a sample image and plotting it 
test_img = image_read("data/face_recognition/brad_pitt/brad_pitt_10.jpg")
test_img <- image_scale(test_img,paste0(width_resized,"x",height_resized,sep =""))
faces <- image_detect_faces(test_img)

face_width = faces$detections$width
face_height = faces$detections$height
face_x = faces$detections$x
face_y = faces$detections$y

face_dim= paste0(face_width,"x",face_height,"+",face_x,"+",face_y)
face_cropped = image_crop(test_img,face_dim)
face_cropped = image_resize(image = face_cropped,paste0(img_size[1],"x",img_size[2]))
face_cropped_arr <- as.integer(face_cropped[[1]])/255
face_cropped_arr <- array_reshape(face_cropped_arr,dim = c(1,img_size,img_channels))
pred <- facenet_model %>% predict(face_cropped_arr)
pred_class <- class_label[which.max(pred)]

test_img <- image_draw(test_img)
rect(xleft = face_x,ybottom = face_y,xright = face_x+ face_width,ytop = face_y+ face_height,lwd = 2,border = "red")
text(face_x,face_y,pred_class,offset = 1,pos = 2,cex = 1.5,col = "pink")
dev.off()

plot(test_img)

png 
  2